In [1]:
import pandas as pd

In [2]:
import numpy as np

In [5]:
data = pd.read_csv("properati_data_fixed_all_years_duplicated_with_outliers.csv", compression = "gzip")

/home/cc/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,7,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
posibles_descripciones_ambientes = {"1 ambiente": 1, "monoambiente": 1, "mono ambiente":1, "2 ambientes": 2,"3 ambientes": 3,
                          "4 ambientes": 4,"5 ambientes": 5, "6 ambientes": 6, "7 ambientes": 7,"8 ambientes": 8,
                          "9 ambientes": 1, "10 ambientes": 10,
                         "ambientes: 1": 1, "ambientes: 2": 2, "ambientes: 3": 3, "ambientes: 4": 4, "ambientes: 5": 5,
                         "ambientes: 6": 6, "ambientes: 7": 7, "un ambiente": 1, "dos ambientes": 2, "tres ambientes": 3,
                         "cuatro ambientes": 4,"cinco ambientes": 5," seis ambientes": 6, "siete ambientes": 7,
                         "2 impecables ambientes": 2, "3 impecables ambientes": 3, "4 amplios ambientes": 4, "2 amplios ambientes": 2,
                          "3 amplios ambientes": 3, "5 amplios ambientes": 5, "2  ambientes": 2, "3  ambientes": 3,
                          "2 grandes ambientes": 2, "6 amplios ambientes": 6, "1 ambientes": 1
                         ,"3 amplios  ambientes": 3, "3 y ½ ambientes": 3.5,"2 y ½ ambientes": 2.5, "2 y medio ambiente": 2.5,
                         "(2) ambientes":2, "2 amplisimos ambientes": 2,"4  ambientes":4,"3 grandes ambientes":3,
                                   "4 ½ ambientes": 4.5, "4 cÃ²modos y confortables ambientes":4, "cantidad de ambientes 4":4,
                                   "cantidad de ambientes 3": 3,"ambientes : 4":4,"ambientes : 5":5,"ambientes : 3":3,"ambientes : 2":2,
                                   "ambientes : 1":1,"chalet de 5  ambientes":5,"4 cÃ³modos y confortables ambientes":4,"4   ambientes":4,
                                   "3 buenos ambientes":3,"4 còmodos y confortables ambientes":4,"2 amb.":2,"3 amb.":3,"4 amb.":4,"1 amb.":1, "5 amb.":5,"6 amb.":6,"7 amb.":7,
                                   "3 (tres) ambientes": 3, "2 (dos) ambientes":2, "4 (cuatro) ambientes":4, "5 (cinco) ambientes": 5, "1 (uno) ambientes": 1, "1 (uno) ambiente":1,
                                   "6 (seis) ambientes":6, "7 (siete) ambientes":7, "8 (ocho) ambientes":8, "9 (nueve) ambientes":9, "10 (diez) ambientes":10, "1 amb":1, "2 amb":2,
                                   "3 amb":3, "4 amb":4, "5 amb":5, "6 amb":6,"7 amb":7,"8 amb":8,"2 ambiente":2,"3 ambiente":3,"4 ambiente":4, "1amb":1,"2amb":2,"3amb":3,"4amb":4,
                                   "5amb":5,"6amb":6,"8amb":8,"7amb":7}


In [108]:

def fix_ambientes(row, search_title, search_extra, search_description):
    
    if pd.isnull(row["rooms"]):
        return np.nan#Ya fueron fixeadas
    cols = []
    if search_description:
        descripcion = row["description"]
        cols.append(descripcion)
    if search_extra:
        extra = row["extra"]
        cols.append(extra)
    if search_title:
        title = row["title"]
        cols.append(title)
        
    ambiente_actual = row["rooms"]
    for col in cols:
        if not(pd.isnull(col)):
            for k in posibles_descripciones_ambientes:
        
                if k in col:
                    value_found = float(posibles_descripciones_ambientes[k])
                    if float(row["rooms"]) != value_found:
                        return value_found
    return row["rooms"]

In [ ]:
data_2017 = data[data["year"] == 2017]

In [ ]:
#Sacamos las de Zona Norte porque no tenemos que predecir casi ninguna de esa zona.
data_2017_zo_zs_caba = data_2017[(data_2017["state_name"] == "Capital Federal") | (data_2017["state_name"] == "Bs.As. G.B.A. Zona Sur") | (data_2017["state_name"] == "Bs.As. G.B.A. Zona Oeste")]

In [ ]:
#Antes vuelvo a aplicar esto xq habia algunos que se pueden corregir
data_2017_zo_zs_caba["rooms"] = data_2017_zo_zs_caba.apply(lambda row: fix_ambientes(row, False, False, True), axis = 1)

In [ ]:
#Los que tienen nulo (room, surface total), (room, cubierta), (cubierta, total) no nos sirven demasiado
len(data_2017_zo_zs_caba)

In [ ]:
data_2017_zo_zs_caba.dropna(axis = 0,how='all', subset=["rooms","surface_total_in_m2"], inplace = True)
data_2017_zo_zs_caba.dropna(axis = 0,how='all', subset=["rooms","surface_covered_in_m2"], inplace = True)
data_2017_zo_zs_caba.dropna(axis = 0,how='all', subset=["surface_covered_in_m2","surface_total_in_m2"], inplace = True)

In [ ]:
data_2017_zo_zs_caba.isnull().sum()

In [ ]:
#A los de rooms nulas tratamos de rellenarlas con con los que tengan superficie total y cubierta parecida (su promedio)

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"].isnull()]["surface_covered_in_m2"].values[4]

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 9]["description"].values[3]

In [ ]:
data_2017_zo_zs_caba["rooms"].value_counts()

In [ ]:
#Con esto vemos que ya mas de 8 ambientes es algo raro, 

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 11]["description"].values[11]

In [ ]:
#Pongo en NaN aquellos que tengan mas de 8 ambientes

In [ ]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 9]["description"].values

In [ ]:
#En promedio por ambiente, cual es su superficie cubierta?
ambs = range(1,9)
for amb in ambs:
    promedio = data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == amb]["surface_covered_in_m2"].mean()
    desviacion = data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == amb]["surface_covered_in_m2"].std()
    print "El promedio para: "+str(amb)+" ambientes es: "+str(promedio)+" y su desviacion estandar es :"+str(desviacion)


In [ ]:
#P/ cada cantidad de ambientes miro su superficie total, superficie cubierta y precio en u$d 
#y elimino las cosas raras

In [ ]:
df = data_2017_zo_zs_caba.copy()

In [ ]:
df[df["rooms"] == 1]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 60 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 1) | ((df["rooms"] == 1) & (df["surface_covered_in_m2"] < 61))]

In [ ]:
df[df["rooms"] == 1]["surface_covered_in_m2"].max()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df[(df["rooms"] == 1) & (df["price_aprox_usd"] == 6809000)]["description"].values[0]

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 2) | ((df["rooms"] == 2) & (df["surface_covered_in_m2"] < 142))]

In [ ]:
len(df)

In [ ]:
df[df["rooms"] == 3]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 3) | ((df["rooms"] == 3) & (df["surface_covered_in_m2"] < 302))]

In [ ]:
df[df["rooms"] == 4]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 4) | ((df["rooms"] == 4) & (df["surface_covered_in_m2"] < 405))]

In [ ]:
df[df["rooms"] == 8]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [ ]:
#Me quedo con las de superficie cubierta < a 80 mts2. Las que tienen mas mts2,
#no podemos saber con exactitud si tiene nincorrecto #rooms o superficie cubierta
df = df[(df["rooms"] != 6) | ((df["rooms"] == 6) & (df["surface_covered_in_m2"] < 1300))]

In [109]:
#Los que tengan mas de 8 ambientes, los pongo como NaN
def filter_ambientes(row):
    ambientes = row["rooms"]
    if ambientes > 8:
        return np.nan
    elif ambientes == 2.5:
        return 3 #Son muy pocos estos dos 
    elif ambientes == 3.5:
        return 4 
    else:
        return ambientes

In [ ]:
df["rooms"] = df.apply(lambda row: filter_ambientes(row), axis = 1)

In [ ]:
df["rooms"].value_counts()

In [ ]:
df["surface_total_in_m2"].value_counts().sort_index(ascending = False).to_dict()

In [45]:
df2 = df.copy()
len(df2)

14166

In [ ]:
df = df2.copy()
df2 = df.copy()

In [ ]:
df["surface_covered_in_m2"].isnull().sum()

In [110]:
def replace_cubierta_nan(row):
    cubierta = row["surface_covered_in_m2"]
    if pd.isnull(cubierta):
        return np.nan
    elif cubierta < 20:
        return np.nan
    elif not(pd.isnull(row["rooms"])):
        rooms = row["rooms"]
        if rooms == 4 and cubierta < 30:
            return np.nan
        elif rooms == 5 and cubierta < 35:
            return np.nan
        elif rooms == 6 and cubierta < 40:
            return np.nan
        elif rooms == 7 and cubierta < 45:
            return np.nan
        elif rooms == 8 and cubierta < 59:
            return np.nan
        else:
            return row["surface_covered_in_m2"]
    else:
        return row["surface_covered_in_m2"]

In [111]:
def replace_total(row):
    sup_total = row["surface_total_in_m2"]
    if sup_total < 20:
        return np.nan
    else:
        return row["surface_total_in_m2"]

In [ ]:
df["surface_total_in_m2"] = df.apply(lambda row: replace_total(row), axis = 1)

In [ ]:
df2["surface_total_in_m2"].isnull().sum()

In [ ]:
df["surface_covered_in_m2"] = df.apply(lambda row: replace_cubierta_nan(row), axis = 1)

In [ ]:
df["surface_covered_in_m2"].isnull().sum()

In [ ]:
df[df["rooms"] == 1]["surface_covered_in_m2"].max()

In [ ]:
df.dropna(axis = 0,how='all', subset=["rooms","surface_total_in_m2"], inplace = True)
df.dropna(axis = 0,how='all', subset=["rooms","surface_covered_in_m2"], inplace = True)
df.dropna(axis = 0,how='all', subset=["surface_covered_in_m2","surface_total_in_m2"], inplace = True)

In [ ]:
#Para cada nulo de rooms, le pongo el promedio de los q tengan su cubierta y total, y para cada
#nulo de cubierta, lo mismo
df.isnull().sum()

In [112]:
def fix_nan_rooms(row, df):
    if not(pd.isnull(row["rooms"])):
        return row["rooms"] #Si ya tenia la cantidad de rooms, esto esta bien
    
    sup_cub = row["surface_covered_in_m2"]
    avg_rooms = np.nan
    dif = 0
    while pd.isnull(avg_rooms):    
        lim_inf = sup_cub - dif
        lim_sup = sup_cub + dif
        act_vecindario = df[df["surface_covered_in_m2"] < lim_sup]
        act_vecindario = df[df["surface_covered_in_m2"] > lim_inf]
        print len(act_vecindario)
        avg_rooms = act_vecindario["rooms"].mean()
        dif += 5
    print "FOUND "+str(avg_rooms)
        

In [ ]:
not_nan_rooms = df[~df["rooms"].isnull()]

In [ ]:
not_nan_rooms["surface_covered_in_m2"].isnull().sum()

In [ ]:
df.corr()

In [ ]:
#Hay una correlacion masomenos alta entre la superficie cubierta y la total, para rellenar la superficie
#cubierta que queda lo hago segun la total

In [ ]:
df2 = df.copy()

In [113]:
#A los que tengan una superficie total < a la cubierta: Les asigno la cubierta
def fix_surface_total_menor_cubierta(row):
    if pd.isnull(row["surface_total_in_m2"]) or pd.isnull(row["surface_covered_in_m2"]):
        return row["surface_total_in_m2"]
    elif row["surface_total_in_m2"] < row["surface_covered_in_m2"]:
        return row["surface_covered_in_m2"]
    else:
        return row["surface_total_in_m2"]

In [ ]:
df.corr()

In [ ]:
df["surface_total_in_m2"] = df.apply(lambda row: fix_surface_total_menor_cubierta(row), axis=1)

In [ ]:
#Hay alguno con superficie total muy grande?
df["surface_total_in_m2"].max()

In [ ]:
df[df["surface_total_in_m2"] == df["surface_total_in_m2"].max()]["description"].values[0]

In [ ]:
#Lo elimino
df[df["surface_total_in_m2"] == df["surface_total_in_m2"].max()]

In [ ]:
df.sort_values(by="surface_total_in_m2", ascending = False)

In [ ]:
df["surface_total_in_m2"].value_counts().sort_index(ascending = False)

In [ ]:
df[df["surface_total_in_m2"] > 2000]["price_aprox_usd"].mean()

In [ ]:
#Elimino propiedades que tengan una superficie total muy alta y un precio muy bajo

In [ ]:
len(df[(df["surface_total_in_m2"] < 2000) | ((df["surface_total_in_m2"] >= 2000) & df["price_aprox_usd"] > 1000000)])

In [ ]:
df = df[(df["surface_total_in_m2"] < 2000) | ((df["surface_total_in_m2"] >= 2000) & df["price_aprox_usd"] > 1000000)]

In [ ]:
df.corr()

In [ ]:
df["surface_total_in_m2"].max()

In [119]:
saved_results = {}

In [114]:
#A partir de la superficie total empiezo a arreglar la sup. cubierta
def fill_superficie_cubierta_con_total(row, df):
    
    if not(pd.isnull(row["surface_covered_in_m2"])):
        return row["surface_covered_in_m2"]
    sup_total = row["surface_total_in_m2"]
    avg_cubierta = np.nan
    dif = 0
    if sup_total in saved_results:
        return saved_results[sup_total]
    while pd.isnull(avg_cubierta):    
        lim_inf = sup_total - dif
        lim_sup = sup_total + dif
        act_vecindario = df[df["surface_total_in_m2"] < lim_sup]
        act_vecindario = act_vecindario[act_vecindario["surface_total_in_m2"] > lim_inf]
        avg_cubierta = act_vecindario["surface_covered_in_m2"].mean()
        dif += 1
        if (dif > 20): #para que sea mas rapido
            dif += 20
    #Guardo el resultado 
    saved_results[row["surface_total_in_m2"]] = avg_cubierta
    return avg_cubierta

In [ ]:
df["surface_covered_in_m2"]= df.apply(lambda row: fill_superficie_cubierta_con_total(row, df), axis = 1)

In [ ]:
print saved_results

In [ ]:
#Por ahora dejamos las propiedades con un valor muy alto, de ultima despues probamos eliminarlas

In [ ]:
saved_surface_total_rooms = {}

In [115]:
def fix_nan_rooms(row, df):
    if not(pd.isnull(row["rooms"])):
        return row["rooms"] #Si ya tenia la cantidad de rooms, esto esta bien
    
    sup_cub = row["surface_covered_in_m2"]
    if sup_cub in saved_surface_total_rooms:
        return saved_surface_total_rooms[sup_cub]
    avg_rooms = np.nan
    dif = 0
    while pd.isnull(avg_rooms):    
        lim_inf = sup_cub - dif
        lim_sup = sup_cub + dif
        act_vecindario = df[df["surface_covered_in_m2"] < lim_sup]
        act_vecindario = df[df["surface_covered_in_m2"] > lim_inf]
        avg_rooms = act_vecindario["rooms"].mean()
        dif += 5
    saved_surface_total_rooms[sup_cub] = avg_rooms
    return avg_rooms
        

In [ ]:
df2 = df.copy()

In [ ]:
df["rooms"] = df.apply(lambda row: fix_nan_rooms(row,df), axis = 1)

In [ ]:
df.corr()

In [ ]:
#Es una casa de 12 millones de dolares con una descripcion muy pobre, la elimino
#Es complicado considerar un vecindario para expensas, a aquella la eliminamos. con floor podemos 
#rellenar con 0's. Ademas en expensas no se ve una correlacion

In [ ]:
df["floor"].fillna(0, inplace = True)

In [ ]:
df.drop(labels = "expenses", axis = 1, inplace = True)

In [116]:
#A las que tengan place_name nulo le asigno su state_name
def fix_null_place_name(row):
    if pd.isnull(row["place_name"]):
        return row["state_name"]
    return row["place_name"]

In [ ]:
df["place_name"] = df.apply(lambda row: fix_null_place_name(row), axis = 1)

In [ ]:
len(df[df["lat-lon"].isnull()]["place_name"].value_counts().index)

In [ ]:
df["lat-lon"].values[0]

In [117]:
#A los de lat-lon nula, busco entre los que tengan el mismo place name y le asigno algun lat lon 
#de ellos
saved_lat_lons = {}
def fix_lat_lon(row, df):
    if not(pd.isnull(row["lat-lon"])):
        return row["lat-lon"]
    place = row["place_name"]
    if place in saved_lat_lons:
        print "FOUND AT SAVED!"
        return saved_lat_lons[place]
    same_place = df[df["place_name"] == place]
    same_place_w_lat_lon = same_place[~same_place["lat-lon"].isnull()]
    try:
        final_pos = same_place_w_lat_lon["lat-lon"].values[0]
    except:
        return np.nan
    saved_lat_lons[place] = final_pos
    return final_pos

In [ ]:
df["lat-lon"] = df.apply(lambda row: fix_lat_lon(row,df), axis = 1)

In [ ]:
#Quedaron 11 nulos, los busco a mano

In [ ]:
df["lat-lon"] = df.apply(lambda row: row["lat-lon"] if not(pd.isnull(row["lat-lon"])) else "-34.800986,-58.218988", axis = 1)

In [ ]:
df.isnull().sum()

In [ ]:
df.to_csv("training_set_filling_nans_con_vecinos.csv", compression ="gzip", index = False)

In [83]:
test_set_inicial = pd.read_csv("properati_testing_noprice_fixed.csv",compression ="gzip", low_memory = False)

In [ ]:
#primero pongo en nuls las cosas raras


In [84]:
test_set_inicial["rooms"] = test_set_inicial.apply(lambda row: filter_ambientes(row), axis = 1)

In [85]:
test_set_inicial["rooms"].value_counts()

2.0    3182
3.0    3071
1.0    2077
4.0    1698
5.0     492
6.0     194
7.0      72
8.0      33
Name: rooms, dtype: int64

In [86]:
len(test_set_inicial)

14166

In [87]:
test_set_inicial["surface_covered_in_m2"].isnull().sum()

1151

In [88]:

test_set_inicial["surface_covered_in_m2"] = test_set_inicial.apply(lambda row: replace_cubierta_nan(row), axis = 1)
test_set_inicial["surface_covered_in_m2"].isnull().sum()

1318

In [89]:
test_set_inicial["surface_total_in_m2"] = test_set_inicial.apply(lambda row: replace_total(row), axis = 1)

In [90]:
test_set_inicial.head(10)

,description,floor,lat-lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2
0,edificio ba houses situado frente al dique 3 d...,NaN,"-34.6109877599,-58.3634635778",Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,departamento,3.0,Capital Federal,NaN,NaN
1,el departamento cuenta con un living-comedor a...,NaN,NaN,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,departamento,NaN,Buenos Aires Interior,NaN,NaN
2,impecable torre coy iii – depa...,NaN,"-34.5893633232,-58.4128798588",Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,departamento,2.0,Capital Federal,48.0,53.0
3,ambiente divisible con pisos d...,NaN,NaN,Chacarita,|Argentina|Capital Federal|Chacarita|,departamento,NaN,Capital Federal,39.0,39.0
4,living comedor con pisos de po...,NaN,NaN,Chacarita,|Argentina|Capital Federal|Chacarita|,departamento,NaN,Capital Federal,51.0,51.0
5,ambiente divisible con pisos d...,NaN,NaN,Chacarita,|Argentina|Capital Federal|Chacarita|,departamento,NaN,Capital Federal,39.0,39.0
6,ambiente divisible con pisos d...,NaN,NaN,Chacarita,|Argentina|Capital Federal|Chacarita|,departamento,NaN,Capital Federal,39.0,39.0
7,ambiente divisible con pisos d...,NaN,NaN,Chacarita,|Argentina|Capital Federal|Chacarita|,departamento,NaN,Capital Federal,39.0,39.0
8,excelente 2 amb- apto profesio...,NaN,"-34.5787213694,-58.4570952614",Colegiales,|Argentina|Capital Federal|Colegiales|,departamento,NaN,Capital Federal,67.0,67.0
9,impecable monoambiente- divisi...,NaN,NaN,Almagro,|Argentina|Capital Federal|Almagro|,departamento,1.0,Capital Federal,47.0,47.0


In [ ]:
#Ademas tengo que chequear monoambientes o 2, 3, .. con superficie cubierta ridicula 

In [91]:
test_set_inicial[(test_set_inicial["rooms"] == 1) & (test_set_inicial["surface_covered_in_m2"] == 1900)]["description"].values[0]

'corredor responsable: uno bienes raices srl - cmcpsi 5821contacto: pablo ortiz - mls id # 420041241-23casa con gran terreno   generoso terreno en  exclusivo  barrio de casas quintas   rodeado de casas de categoria  .    tiene 1900 m2    a\xc3\xb1osa arboleda y frutales   ideal para vivienda permanenete o para fin de semana     con todas las comodidades para pasar un fin de semana  comodo  alambrado perimetral + porton.  muy seguro - tienen vecinos con viviendas permanentes   si le  intereso  no deje de consultarnos en el formulario .  en breve lo contactaremos!!   .      '

In [92]:
test_set_inicial[(test_set_inicial["rooms"] == 1) & (test_set_inicial["surface_covered_in_m2"] > 100)]

,description,floor,lat-lon,place_name,place_with_parent_names,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2
178,hermoso ph tipo loft recien reciclado. si se d...,NaN,NaN,Monserrat,|Argentina|Capital Federal|Monserrat|,ph,1.0,Capital Federal,133.0,169.0
1539,corredor responsable: uno bienes raices srl - ...,NaN,"-34.574147,-58.848867",Moreno,|Argentina|Bs.As. G.B.A. Zona Oeste|Moreno|Mor...,casa,1.0,Bs.As. G.B.A. Zona Oeste,1900.0,1900.0
1676,"moderna casa, de solo 10 aÑos, con muy buenas ...",NaN,"-34.6382715,-58.4869238",Villa Luro,|Argentina|Capital Federal|Villa Luro|,casa,1.0,Capital Federal,170.0,290.0
1680,dueÑo directo-inmobiliarias abstenerse- espect...,NaN,"-34.5843717,-58.4221871",Palermo,|Argentina|Capital Federal|Palermo|,departamento,1.0,Capital Federal,160.0,215.0
1682,dueÑo directo-inmobiliarias abstenerse- espect...,NaN,"-34.5843717,-58.4221871",Palermo,|Argentina|Capital Federal|Palermo|,departamento,1.0,Capital Federal,160.0,215.0
2379,ideal consultorio oficinas apto profesionaldob...,NaN,"-34.62084198,-58.37721252",San Telmo,|Argentina|Capital Federal|San Telmo|,departamento,1.0,Capital Federal,4733.0,60.0
2668,hermoso chalet a 2 aguas de 112 m2 sobre lote ...,NaN,"-34.6009407,-58.68854141",Villa Udaondo,|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|...,casa,1.0,Bs.As. G.B.A. Zona Oeste,122.0,600.0
2765,casa a 8 cuadras de la estación muy buen estad...,NaN,"-34.654953,-58.7814064",Moreno,|Argentina|Bs.As. G.B.A. Zona Oeste|Moreno|Mor...,casa,1.0,Bs.As. G.B.A. Zona Oeste,135.0,135.0
3013,lote con 2 departamentos a demoler medidas; 8....,NaN,"-34.65138626,-58.61279297",Morón,|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|Morón|,casa,1.0,Bs.As. G.B.A. Zona Oeste,190.0,225.0
3069,vendo casa sobre 2 lotes con salida a dos cal...,NaN,"-34.66523743,-58.82032776",Moreno,|Argentina|Bs.As. G.B.A. Zona Oeste|Moreno|Mor...,casa,1.0,Bs.As. G.B.A. Zona Oeste,110.0,695.0


In [93]:
#En promedio por ambiente, cual es su superficie cubierta para el test?
ambs = range(1,9)
for amb in ambs:
    promedio = test_set_inicial[test_set_inicial["rooms"] == amb]["surface_covered_in_m2"].mean()
    desviacion = test_set_inicial[test_set_inicial["rooms"] == amb]["surface_covered_in_m2"].std()
    print "El promedio para: "+str(amb)+" ambientes es: "+str(promedio)+" y su desviacion estandar es :"+str(desviacion)
#se ve que para propiedades de 5 ambientes la sup es demaciado grande comparado a los varores de ambientes mayores

El promedio para: 1 ambientes es: 42.4656762295 y su desviacion estandar es :116.531818117
El promedio para: 2 ambientes es: 56.176410605 y su desviacion estandar es :105.322628062
El promedio para: 3 ambientes es: 98.7613836922 y su desviacion estandar es :264.302941219
El promedio para: 4 ambientes es: 226.727684347 y su desviacion estandar es :3761.5428175
El promedio para: 5 ambientes es: 7159.68376068 y su desviacion estandar es :149909.531615
El promedio para: 6 ambientes es: 234.983695652 y su desviacion estandar es :114.483610183
El promedio para: 7 ambientes es: 261.485714286 y su desviacion estandar es :113.212477273
El promedio para: 8 ambientes es: 289.0625 y su desviacion estandar es :141.127701837


In [94]:
df = test_set_inicial.copy()

In [95]:
df[df["rooms"] == 1]["surface_covered_in_m2"].value_counts().sort_index(ascending = False).to_dict()

{20.0: 11,
 21.0: 12,
 22.0: 18,
 23.0: 24,
 24.0: 32,
 25.0: 55,
 26.0: 65,
 27.0: 60,
 28.0: 82,
 29.0: 31,
 30.0: 136,
 31.0: 63,
 32.0: 115,
 33.0: 112,
 34.0: 151,
 35.0: 96,
 36.0: 94,
 37.0: 73,
 38.0: 57,
 39.0: 56,
 40.0: 79,
 41.0: 45,
 42.0: 52,
 43.0: 19,
 44.0: 67,
 45.0: 55,
 46.0: 48,
 47.0: 20,
 48.0: 30,
 49.0: 10,
 50.0: 24,
 51.0: 4,
 52.0: 21,
 53.0: 4,
 54.0: 5,
 55.0: 12,
 56.0: 4,
 57.0: 1,
 58.0: 2,
 59.0: 2,
 60.0: 13,
 61.0: 3,
 62.0: 1,
 63.0: 1,
 64.0: 1,
 65.0: 2,
 66.0: 3,
 67.0: 2,
 68.0: 1,
 69.0: 2,
 70.0: 4,
 73.0: 2,
 74.0: 1,
 75.0: 2,
 76.0: 1,
 80.0: 1,
 81.0: 4,
 87.0: 1,
 90.0: 15,
 97.0: 2,
 98.0: 1,
 100.0: 9,
 110.0: 1,
 120.0: 3,
 122.0: 1,
 133.0: 2,
 135.0: 1,
 136.0: 1,
 139.0: 1,
 143.0: 2,
 151.0: 1,
 159.0: 1,
 160.0: 2,
 165.0: 2,
 170.0: 1,
 180.0: 1,
 186.0: 1,
 190.0: 2,
 206.0: 1,
 220.0: 2,
 226.0: 1,
 235.0: 1,
 305.0: 1,
 385.0: 1,
 388.0: 1,
 1900.0: 1,
 4733.0: 1}

In [96]:
#en caso que haya una cantidad mayor  a 8 amb se coloca nan se considera outlier 
df["rooms"] = df.apply(lambda row: filter_ambientes(row), axis = 1)
len(df)

14166

In [97]:
df2 = df.copy()
len(df2)

14166

In [98]:
#vemos cuantas filas tienen superficie cubierta nula
df["surface_covered_in_m2"].isnull().sum()

1318

In [99]:
df["surface_total_in_m2"] = df.apply(lambda row: replace_total(row), axis = 1)

In [100]:
df["surface_covered_in_m2"] = df.apply(lambda row: replace_cubierta_nan(row), axis = 1)

In [75]:
#aca le saco filas al df
df.dropna(axis = 0,how='all', subset=["rooms","surface_total_in_m2"], inplace = True)
df.dropna(axis = 0,how='all', subset=["rooms","surface_covered_in_m2"], inplace = True)
df.dropna(axis = 0,how='all', subset=["surface_covered_in_m2","surface_total_in_m2"], inplace = True)
len(df)

12387

In [101]:
df.isnull().sum()

description                    0
floor                      11561
lat-lon                     3679
place_name                   264
place_with_parent_names        0
property_type                  0
rooms                       3347
state_name                     0
surface_covered_in_m2       1318
surface_total_in_m2         2310
dtype: int64

In [105]:
df["surface_total_in_m2"] = df.apply(lambda row: fix_surface_total_menor_cubierta(row), axis=1)
df.surface_total_in_m2.isnull().sum()

2310

In [106]:
#Hay alguno con superficie total muy grande?
df["surface_total_in_m2"].max()

148000.0

In [ ]:
df["surface_covered_in_m2"]= df.apply(lambda row: fill_superficie_cubierta_con_total(row, df), axis = 1)


In [ ]:
df.isnull().sum()

In [ ]:
df.head(1)

In [ ]:
df["rooms"] = df.apply(lambda row: fix_nan_rooms(row,df), axis = 1)

In [ ]:
df["floor"].fillna(0, inplace = True)

In [ ]:
df["place_name"] = df.apply(lambda row: fix_null_place_name(row), axis = 1)

In [ ]:
df["lat-lon"] = df.apply(lambda row: fix_lat_lon(row,df), axis = 1)

In [ ]:
df["lat-lon"] = df.apply(lambda row: row["lat-lon"] if not(pd.isnull(row["lat-lon"])) else "-34.800986,-58.218988", axis = 1)

In [ ]:
df.isnull().sum()

In [ ]:
df.to_csv("test_set_filling_nans_con_vecinos.csv", compression ="gzip", index = False)